In [1]:
import numpy as np
import pandas as pd

import os

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("EDA") \
    .getOrCreate()

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1599697564776'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '33455'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'EDA'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '192.168.2.162')]

In [4]:
data_dir = "../MLE_Project_Data"
data_file_name = "train_data.csv"
data_path = os.path.join(data_dir,data_file_name)

In [14]:
raw_df = spark.read.load(data_path,format="csv",delimiter="|",header=True)

In [17]:
print(raw_df.count(),len(raw_df.columns))

41907133 36


In [33]:
raw_df.head(2).toPandas()

AttributeError: 'list' object has no attribute 'toPandas'

In [ ]:
raw_df.tail(2).toPandas()

In [22]:
raw_df.schema

StructType(List(StructField(label,StringType,true),StructField(uid,StringType,true),StructField(task_id,StringType,true),StructField(adv_id,StringType,true),StructField(creat_type_cd,StringType,true),StructField(adv_prim_id,StringType,true),StructField(dev_id,StringType,true),StructField(inter_type_cd,StringType,true),StructField(slot_id,StringType,true),StructField(spread_app_id,StringType,true),StructField(tags,StringType,true),StructField(app_first_class,StringType,true),StructField(app_second_class,StringType,true),StructField(age,StringType,true),StructField(city,StringType,true),StructField(city_rank,StringType,true),StructField(device_name,StringType,true),StructField(device_size,StringType,true),StructField(career,StringType,true),StructField(gender,StringType,true),StructField(net_type,StringType,true),StructField(residence,StringType,true),StructField(his_app_size,StringType,true),StructField(his_on_shelf_time,StringType,true),StructField(app_score,StringType,true),StructFiel

In [31]:
describe = raw_df.describe()

In [38]:
describe.toPandas().set_index("summary").T

summary,count,mean,stddev,min,max
label,41907133,0.034492648304048856,0.18249083898022259,0,1
uid,41907133,1618366.5450654665,357389.65521592135,1000001,2237673
task_id,41907133,3436.3739258660335,1429.9486235761321,1001,5992
adv_id,41907133,3963.51341013951,1720.5691813375768,1001,7020
creat_type_cd,41907133,6.490081748135813,1.231181612514361,2,9
adv_prim_id,41907133,159.31912006483478,30.899225966287723,101,214
dev_id,41907133,41.59357343772479,17.41486835377897,11,72
inter_type_cd,41907133,4.646959504483402,0.7103276382288628,2,5
slot_id,41907133,15.254736109005595,3.167097985067854,11,22
spread_app_id,41907133,51.9989373646725,25.10112386162042,11,89


# Loading Data

Pandas cannot handel 6G data while simply reading it.
There are four solutions I can think of:
1. reading only a few columns and analyze at once. However, calculation such as correlation might be a problem in the future even I successfully read in the data.
2. specify each dtype since default dtype takes more memories. However it might have future problem asa point 1.
3. load data in as matrix
4. use spark

Of all the solutioins above, I will try second method first just to test if I can load in the data this way and I am just so familiar analyzing data using pandas, can't give it up so quick without giving it a shot.
However, I think it makes more sense to use spark for following reasons:
1. It can ba a lot faster when doing calculation.
2. The dataset I have only cover 7 days of behavoior data, In the real world setting, I think this type of data is the classic "big data" people are talking all the time. And spark is designed for handling this type of data! 
3. Because the reason in bullet point 2, even I am not dealing with sreaming data, It is better to showcase I have this skillset to the employers.

In [ ]:
raw_df = pd.DataFrame()
for idx,df in enumerate(pd.read_csv(os.path.join(data_dir,data_file_name),sep="|",chunksize=100000)):
    print(idx)
    raw_df = pd.concat((raw_df,df),axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189


In [ ]:
raw_df.head()

In [ ]:
raw_df.head()